In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, VBox, HBox, Button, Layout, Dropdown, interactive_output, Output

In [3]:
%pip install -q ipywidgets ipympl

Note: you may need to restart the kernel to use updated packages.


In [10]:
# === Interactive 1D Perceptron (robust, backend-agnostic) ===
# Works without %matplotlib widget. Uses ipywidgets + Output to re-render the figure.

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, VBox, HBox, Button, Layout, Dropdown, interactive_output, Output
from IPython.display import display, clear_output

# --- Data generation ---
def make_data(n_per_class=60, sep=2.5, seed=0):
    rng = np.random.default_rng(seed)
    x0 = rng.normal(loc=-sep/2, scale=1.0, size=n_per_class)
    x1 = rng.normal(loc= sep/2, scale=1.0, size=n_per_class)
    x = np.concatenate([x0, x1])
    y = np.concatenate([np.zeros_like(x0, dtype=int), np.ones_like(x1, dtype=int)])
    y_plot = y + rng.normal(0, 0.03, size=y.shape)  # small vertical jitter for visibility
    return x, y, y_plot

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# --- Initial dataset ---
x, y, y_plot = make_data(sep=2.5, seed=0)

# --- Widgets ---
w    = FloatSlider(value=1.0, min=-10, max=10, step=0.01, description="w",   layout=Layout(width='300px'), continuous_update=True)
b    = FloatSlider(value=0.0, min=-10, max=10, step=0.01, description="b",   layout=Layout(width='300px'), continuous_update=True)
sep  = FloatSlider(value=2.5, min=0.0, max=6.0, step=0.1, description="sep", layout=Layout(width='300px'), continuous_update=False)
seed = IntSlider(value=0, min=0, max=9999, step=1, description="seed",       layout=Layout(width='300px'), continuous_update=False)
mode = Dropdown(options=[("Sigmoid + 0.5", "sigmoid"), ("Hard step", "step")],
                value="sigmoid", description="mode", layout=Layout(width='220px'))
regen = Button(description="Regenerate data", layout=Layout(width='160px'))

out = Output()

def predict_labels(x_, w_, b_, mode_):
    z = w_ * x_ + b_
    if mode_ == "sigmoid":
        return (sigmoid(z) >= 0.5).astype(int)
    else:
        return (z >= 0).astype(int)

def draw(w, b, mode):
    """Rebuild the figure each time to avoid backend issues."""
    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(7.5, 4.5))

        # Predictions/metrics
        yhat = predict_labels(x, w, b, mode)
        acc = (yhat == y).mean()

        # Scatter
        ax.scatter(x[y==0], y_plot[y==0], label="class 0", alpha=0.85)
        ax.scatter(x[y==1], y_plot[y==1], label="class 1", alpha=0.85)

        # Decision boundary x = -b/w (if defined)
        if abs(w) > 1e-8:
            xb = -b / w
            ax.axvline(xb, ls="--", lw=2, label="decision boundary x=-b/w")

        ax.set_title(f"1D Perceptron ({'sigmoid' if mode=='sigmoid' else 'step'}): "
                     f"accuracy={acc*100:.1f}% | w={w:.2f}, b={b:.2f}")
        ax.set_xlabel("feature x")
        ax.set_ylabel("class (with jitter)")
        ax.grid(True, alpha=0.3)
        ax.legend(loc="best")
        plt.show()

def on_regen_clicked(_):
    global x, y, y_plot
    x, y, y_plot = make_data(sep=sep.value, seed=seed.value)
    # Trigger a redraw with current w/b/mode
    draw(w.value, b.value, mode.value)

# Wire interactivity
regen.on_click(on_regen_clicked)
io = interactive_output(draw, {"w": w, "b": b, "mode": mode})

# Layout + initial render
display(VBox([
    HBox([w, b]),
    HBox([sep, seed, regen]),
    mode,
    out
]))
draw(w.value, b.value, mode.value)


In [11]:


# --- Data generation (2D) ---
def make_data_2d(n_per_class=80, sep=3.0, seed=0):
    """
    Two 2D Gaussian blobs:
      class 0 ~ N([ -sep/2, 0], I)
      class 1 ~ N([ +sep/2, 0], I)
    """
    rng = np.random.default_rng(seed)
    mean0 = np.array([-sep/2.0, 0.0])
    mean1 = np.array([ sep/2.0, 0.0])
    cov = np.eye(2)

    x0 = rng.multivariate_normal(mean0, cov, size=n_per_class)
    x1 = rng.multivariate_normal(mean1, cov, size=n_per_class)
    x = np.vstack([x0, x1])
    y = np.concatenate([np.zeros(n_per_class, dtype=int), np.ones(n_per_class, dtype=int)])
    return x, y

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# --- Initial dataset ---
x, y = make_data_2d(sep=3.0, seed=0)

# --- Widgets ---
w1   = FloatSlider(value=1.0, min=-10, max=10, step=0.01, description="w1",  layout=Layout(width='300px'), continuous_update=True)
w2   = FloatSlider(value=0.5, min=-10, max=10, step=0.01, description="w2",  layout=Layout(width='300px'), continuous_update=True)
b    = FloatSlider(value=0.0, min=-10, max=10, step=0.01, description="b",   layout=Layout(width='300px'), continuous_update=True)
sep  = FloatSlider(value=3.0, min=0.0, max=8.0, step=0.1, description="sep", layout=Layout(width='300px'), continuous_update=False)
seed = IntSlider(value=0, min=0, max=9999, step=1, description="seed",       layout=Layout(width='300px'), continuous_update=False)
mode = Dropdown(options=[("Sigmoid + 0.5", "sigmoid"), ("Hard step", "step")],
                value="sigmoid", description="mode", layout=Layout(width='220px'))
regen = Button(description="Regenerate data", layout=Layout(width='160px'))

out = Output()

def predict_labels(x_, w1_, w2_, b_, mode_):
    z = w1_ * x_[:, 0] + w2_ * x_[:, 1] + b_
    if mode_ == "sigmoid":
        return (sigmoid(z) >= 0.5).astype(int)
    else:
        return (z >= 0).astype(int)

def draw(w1_val, w2_val, b_val, mode_val):
    """Rebuild the 2D figure each time to avoid backend issues."""
    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(7.5, 6.0))

        # Predictions/metrics
        yhat = predict_labels(x, w1_val, w2_val, b_val, mode_val)
        acc = (yhat == y).mean()

        # Scatter by class
        ax.scatter(x[y==0, 0], x[y==0, 1], label="class 0", alpha=0.85)
        ax.scatter(x[y==1, 0], x[y==1, 1], label="class 1", alpha=0.85)

        # Decision boundary: w1*x1 + w2*x2 + b = 0
        # If |w2|>0, draw as line x2 = -(w1*x1 + b)/w2 across current x-lims.
        # Else if |w1|>0, draw vertical line x1 = -b/w1.
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        eps = 1e-9
        if abs(w2_val) > eps:
            xs = np.linspace(xlim[0], xlim[1], 200)
            ys = -(w1_val * xs + b_val) / w2_val
            ax.plot(xs, ys, ls="--", lw=2, label="decision boundary")
        elif abs(w1_val) > eps:
            xb = -b_val / w1_val
            ax.axvline(xb, ls="--", lw=2, label="decision boundary")

        ax.set_title(
            f"2D Perceptron ({'sigmoid' if mode_val=='sigmoid' else 'step'}): "
            f"accuracy={acc*100:.1f}% | w=({w1_val:.2f}, {w2_val:.2f}), b={b_val:.2f}"
        )
        ax.set_xlabel("x1")
        ax.set_ylabel("x2")
        ax.grid(True, alpha=0.3)
        ax.legend(loc="best")
        plt.show()

def on_regen_clicked(_):
    global x, y
    x, y = make_data_2d(sep=sep.value, seed=seed.value)
    # Trigger redraw with current params
    draw(w1.value, w2.value, b.value, mode.value)

# Wire interactivity
regen.on_click(on_regen_clicked)
io = interactive_output(draw, {"w1_val": w1, "w2_val": w2, "b_val": b, "mode_val": mode})

# Layout + initial render
display(VBox([
    HBox([w1, w2, b]),
    HBox([sep, seed, regen]),
    mode,
    out
]))
draw(w1.value, w2.value, b.value, mode.value)
